In [1]:
pip install skyfield matplotlib cartopy requests plotly pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 11.0 MB/s eta 0:00:00


In [25]:
import plotly.graph_objects as go
import numpy as np
from datetime import datetime
import requests
from skyfield.api import load, wgs84, EarthSatellite


In [26]:
def get_satellite_position(norad_id):
    # Obtener los datos TLE más recientes de SatNOGS
    url = f"https://db.satnogs.org/api/tle/?norad_cat_id={norad_id}&format=json"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error al obtener datos TLE: HTTP {response.status_code}")
        return None

    tle_data = response.json()

    if not tle_data:
        print(f"No se encontraron datos TLE para el satélite con NORAD ID {norad_id}")
        return None

    # Extraer las líneas TLE
    tle1 = tle_data[0]['tle1']
    tle2 = tle_data[0]['tle2']

    # Crear un objeto satélite usando Skyfield
    ts = load.timescale()
    satellite = EarthSatellite(tle1, tle2, tle_data[0]['tle0'], ts)

    # Obtener la posición actual
    t = ts.now()
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)

    # Obtener latitud, longitud y altitud
    lat = subpoint.latitude.degrees
    lon = subpoint.longitude.degrees
    elevation = subpoint.elevation.km

    # Obtener la hora actual
    current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

    return {
        'time': current_time,
        'latitude': lat,
        'longitude': lon,
        'altitude': elevation,
        'satellite_name': satellite.name
    }

# Ejemplo de uso
norad_id = "98851"  # Este es el ID del satélite LEMU Nge
position = get_satellite_position(norad_id)

if position:
    print(f"Posición del satélite {position['satellite_name']} (NORAD ID: {norad_id}):")
    print(f"Tiempo: {position['time']}")
    print(f"Latitud: {position['latitude']:.6f}°")
    print(f"Longitud: {position['longitude']:.6f}°")
    print(f"Altitud: {position['altitude']:.2f} km")

Posición del satélite LEMU-NGE (NORAD ID: 98851):
Tiempo: 2024-08-23 14:30:59 UTC
Latitud: -21.725657°
Longitud: 124.937958°
Altitud: 605.89 km


In [27]:
import plotly.graph_objects as go
import numpy as np
from skyfield.api import load
from skyfield.api import EarthSatellite
from datetime import datetime
import requests

# ... (tu función get_satellite_position)

def visualize_satellite_position(position):
    # Crear los datos para el globo terráqueo
    def sphere(size, color):
        phi = np.linspace(0, np.pi, 100)
        theta = np.linspace(0, 2*np.pi, 100)
        x = size * np.outer(np.sin(theta), np.cos(phi))
        y = size * np.outer(np.sin(theta), np.sin(phi))
        z = size * np.outer(np.cos(theta), np.ones_like(phi))
        return go.Surface(x=x, y=y, z=z, colorscale=[[0, color], [1, color]])

    # Crear la figura
    fig = go.Figure(data=[sphere(1, 'lightblue')])


    # Añadir el satélite
    lat = position['latitude']
    lon = position['longitude']
    alt = position['altitude']

    # Convertir lat, lon a coordenadas cartesianas
    r = 1 + alt/6371  # Radio normalizado (6371 km es el radio aproximado de la Tierra)
    x = r * np.cos(np.radians(lat)) * np.cos(np.radians(lon))
    y = r * np.cos(np.radians(lat)) * np.sin(np.radians(lon))
    z = r * np.sin(np.radians(lat))


    fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z], mode='markers', marker=dict(size=5, color='red')))

    # Configurar el diseño
    fig.update_layout(
        title=f"{position['satellite_name']} Position",
        scene=dict(
            xaxis_title='',
            yaxis_title='',
            zaxis_title='',
            aspectmode='data'
        ),
        annotations=[
            dict(
                x=0.0,
                y=0.0,
                xref='paper',
                yref='paper',
                text=f'Lat: {lat:.2f}°, Lon: {lon:.2f}°, Alt: {alt:.2f} km<br>Time: {position["time"]}',
                showarrow=False
            )
        ]
    )

    # Mostrar la figura
    fig.show()

# Ejemplo de uso
norad_id = "98851"  # Este es el ID del Lemu Nge
position = get_satellite_position(norad_id)

if position:
    visualize_satellite_position(position)

In [22]:
def get_satellite_position(tle1, tle2):
    # Crear un objeto satélite usando Skyfield
    ts = load.timescale()
    satellite = EarthSatellite(tle1, tle2, "LEMU NGE", ts)

    # Obtener la posición actual
    t = ts.now()
    geocentric = satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)

    # Obtener latitud, longitud y altitud
    lat = subpoint.latitude.degrees
    lon = subpoint.longitude.degrees
    elevation = subpoint.elevation.km

    # Obtener la hora actual
    current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

    return {
        'time': current_time,
        'latitude': lat,
        'longitude': lon,
        'altitude': elevation,
        'satellite_name': satellite.name
    }

# Usar los datos TLE proporcionados
tle1 = "1 25544U 98067A   24235.67560383  .00033288  00000+0  57656-3 0  9999"
tle2 = "2 25544  51.6399 345.9258 0004816 238.7404 121.3114 15.50586304468860"

position = get_satellite_position(tle1, tle2)

# Crear el globo terráqueo
fig = go.Figure(data=go.Scattergeo(
    lon = [position['longitude']],
    lat = [position['latitude']],
    mode = 'markers',
    marker = dict(
        size = 12,
        color = 'red',
        symbol = 'star'
    ),
    text = f"{position['satellite_name']}<br>Altitud: {position['altitude']:.2f} km",
    hoverinfo = 'text'
))

fig.update_layout(
    title = f"Posición de {position['satellite_name']} en {position['time']}",
    geo = dict(
        projection_type = 'orthographic',
        showland = True,
        showcountries = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        showocean = True,
        oceancolor = 'rgb(217, 240, 255)'
    )
)

# Mostrar el gráfico
fig.show()

In [14]:
import plotly.graph_objects as go
import numpy as np

def visualize_satellite_position(position):
    # Crear los datos para el globo terráqueo con una capa cartográfica
    def earth_globe(size):
        # Definir la malla para la esfera
        phi = np.linspace(0, np.pi, 100)
        theta = np.linspace(0, 2*np.pi, 100)
        x = size * np.outer(np.sin(theta), np.cos(phi))
        y = size * np.outer(np.sin(theta), np.sin(phi))
        z = size * np.outer(np.cos(theta), np.ones_like(phi))

        # Usar una textura de mapa de la Tierra
        return go.Surface(
            x=x, y=y, z=z,
            surfacecolor=np.flipud(np.outer(np.linspace(-90, 90, 100), np.ones(100))),  # Color basado en latitud
            colorscale='Earth',  # Usar un colorescale que simula el mapa terrestre
            cmin=-90, cmax=90,
            showscale=False
        )

    # Crear la figura con la capa cartográfica de la Tierra
    fig = go.Figure(data=[earth_globe(1)])

    # Añadir el satélite
    lat = position['latitude']
    lon = position['longitude']
    alt = position['altitude']

    # Convertir lat, lon a coordenadas cartesianas
    r = 1 + alt / 6371  # Radio normalizado (6371 km es el radio aproximado de la Tierra)
    x = r * np.cos(np.radians(lat)) * np.cos(np.radians(lon))
    y = r * np.cos(np.radians(lat)) * np.sin(np.radians(lon))
    z = r * np.sin(np.radians(lat))

    fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z], mode='markers', marker=dict(size=5, color='red')))

    # Configurar el diseño
    fig.update_layout(
        title=f"{position['satellite_name']} Position",
        scene=dict(
            xaxis_title='',
            yaxis_title='',
            zaxis_title='',
            aspectmode='data',
            xaxis=dict(showbackground=False),
            yaxis=dict(showbackground=False),
            zaxis=dict(showbackground=False)
        ),
        annotations=[
            dict(
                x=0.0,
                y=0.0,
                xref='paper',
                yref='paper',
                text=f'Lat: {lat:.2f}°, Lon: {lon:.2f}°, Alt: {alt:.2f} km<br>Time: {position["time"]}',
                showarrow=False
            )
        ]
    )

    # Mostrar la figura
    fig.show()

# Ejemplo de uso
position = {
    'satellite_name': 'ISS (Zarya)',
    'latitude': 45.0,
    'longitude': -122.0,
    'altitude': 400,  # km
    'time': '2024-08-23T12:34:56Z'
}

visualize_satellite_position(position)


In [28]:
def visualize_satellite_position(position):
    # Crear los datos para el globo terráqueo
    def sphere(size, color):
        phi = np.linspace(0, np.pi, 100)
        theta = np.linspace(0, 2*np.pi, 100)
        x = size * np.outer(np.sin(theta), np.cos(phi))
        y = size * np.outer(np.sin(theta), np.sin(phi))
        z = size * np.outer(np.cos(theta), np.ones_like(phi))
        return go.Surface(x=x, y=y, z=z, colorscale=[[0, color], [1, color]])

    # Crear la figura 3D
    fig = go.Figure(data=[sphere(1, 'lightblue')])

    # Añadir el satélite
    lat = position['latitude']
    lon = position['longitude']
    alt = position['altitude']

    # Convertir lat, lon a coordenadas cartesianas
    r = 1 + alt / 6371  # Radio normalizado (6371 km es el radio aproximado de la Tierra)
    x = r * np.cos(np.radians(lat)) * np.cos(np.radians(lon))
    y = r * np.cos(np.radians(lat)) * np.sin(np.radians(lon))
    z = r * np.sin(np.radians(lat))

    fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z], mode='markers', marker=dict(size=5, color='red')))

    # Configurar el diseño del globo terráqueo
    fig.update_layout(
        title=f"{position['satellite_name']} Position",
        scene=dict(
            xaxis_title='',
            yaxis_title='',
            zaxis_title='',
            aspectmode='data'
        ),
        annotations=[
            dict(
                x=0.0,
                y=0.0,
                xref='paper',
                yref='paper',
                text=f'Lat: {lat:.2f}°, Lon: {lon:.2f}°, Alt: {alt:.2f} km<br>Time: {position["time"]}',
                showarrow=False
            )
        ]
    )

    # Crear proyección ortográfica
    fig_geo = go.Figure(go.Scattergeo(
        lon=[lon],
        lat=[lat],
        mode='markers',
        marker=dict(size=8, color='red'),
    ))

    fig_geo.update_geos(
        projection_type="orthographic",
        showcountries=True, countrycolor="RebeccaPurple"
    )

    fig_geo.update_layout(
        title=f"Orthographic Projection of {position['satellite_name']}",
        margin={"r":0,"t":60,"l":0,"b":50}
    )

    # Mostrar las figuras
    fig.show()
    fig_geo.show()

# Usando la función get_satellite_position que definimos anteriormente
norad_id = "98851"  # Lemu Nge
position = get_satellite_position(norad_id)
if position:
    visualize_satellite_position(position)
